### ENSF 612 Final Project
#### IMDB-Sentiment Analysis of 2022 Movies based on User-Reviews

##### Group Members:
- Samuel Sofela
- Sam Rainbow
- Christopher DiMattia

This document shows the NLP machine learning workflow for evaluating models on movie reviews that were featurized using bag of words.

In [0]:
# Download modules
%pip install nltk

Python interpreter will be restarted.
Python interpreter will be restarted.


### Import modules

In [0]:
import nltk
from pyspark.sql import SparkSession
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords
stop = stopwords.words('english')
spark = SparkSession.builder.appName("612_Proj").config("spark.task.cpus", "2").getOrCreate()
from nltk.tokenize import word_tokenize
from pyspark.sql.functions import udf, split, col
from pyspark.sql.types import ArrayType, StringType
from pyspark.ml.feature import CountVectorizer
from nltk.stem import WordNetLemmatizer
from pyspark.sql.functions import concat
from pyspark.sql.functions import concat_ws
from nltk.stem import SnowballStemmer
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import col
from pyspark.sql.types import StructType, StructField, StringType


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


### Loading the csv file containing the movie review data.

In [0]:
# Loads a CSV file into a dataframe
def csv_to_df(fname):
    
    #Location of dbfs
    dbfs_loc = "dbfs:/FileStore/shared_uploads/samuel.sofela@ucalgary.ca/"
    filename_complete = dbfs_loc + fname
    filetype="csv"
    
    #Options for loading
    inf_sch = "true"
    is_header = "true"
    delim = ","
    multiline = "true"
    escape = "\""
    
    # Load the data into a dataframe using options above
    df = spark.read.format(filetype)\
                .option("header", is_header)\
                .option("inferSchema", inf_sch)\
                .option("multiline", multiline)\
                .option("escape", escape).load(filename_complete)
    
    return df

In [0]:
# Load CSVs
all_reviews = csv_to_df("IMDB_dataset_rated_final.csv")

In [0]:
#concatenate review title and review content with space inbetween
all_reviews = all_reviews.withColumn("Complete_Content", concat_ws(" ", "Review Title", "Review Content"))

### Data Preprocesssing

In [0]:
# Function to clean text and convert all letters to lower case
def clean_text(text):
    text = ''.join(c for c in text if c.isalpha() or c.isspace())
    text = text.lower()
    return text

# Function to remove stop words and tokennize the review
def remove_stopwords(text):
    stop = set(nltk.corpus.stopwords.words('english'))
    tokens = word_tokenize(text)
    filtered_tokens = [token for token in tokens if token.lower() not in stop]
    
    return filtered_tokens

# Function to lemmatize tokens with POS
def lemmatize_text_pos(filtered_tokens):
    lemmatizer = WordNetLemmatizer()
    pos_tagged = nltk.pos_tag(filtered_tokens)
    lemmatized_tokens = [lemmatizer.lemmatize(token, get_wordnet_pos(pos)) if get_wordnet_pos(pos) else token for token, pos in pos_tagged]
    return lemmatized_tokens

# Funtion to apply Snowball stemming
def stemmer_snowball(filtered_tokens):
    snowball = SnowballStemmer(language = 'english')
    print(filtered_tokens)
    stemmed_words = [snowball.stem(word) for word in filtered_tokens]
    return stemmed_words

# Helper function to get the WordNet POS tag from the NLTK POS tag
def get_wordnet_pos(nltk_pos_tag):
    if nltk_pos_tag.startswith('J'):
        return 'a'
    elif nltk_pos_tag.startswith('V'):
        return 'v'
    elif nltk_pos_tag.startswith('N'):
        return 'n'
    elif nltk_pos_tag.startswith('R'):
        return 'r'
    else:
        return None

# Function that applies the cleaning, stop word removal, and lemmatization functions
def preprocess_text(text):
    cleaned = clean_text(text)
    filtered = remove_stopwords(cleaned)
    lemmatized = lemmatize_text_pos(filtered)
    print("this is sam")
    return lemmatized

#Function applies cleaning, stop word removal and snowball stemming functions
def preprocess_text_stemming(text):
    cleaned = clean_text(text)
    filtered = remove_stopwords(cleaned)
    stemmed = stemmer_snowball(filtered)
    return stemmed

# Creating UDF for Preprocessing text using lemmatization
preprocess_text_udf = udf(preprocess_text, ArrayType(StringType()))

# Creating UDF Preprocessing text using snowball stemming
preprocess_text_udf_stemmed = udf(preprocess_text_stemming, ArrayType(StringType()))

# Apply the UDF to the "Review Content" column and create a new columns for lemmatization and stemming
df = all_reviews.withColumn("lemmatized_text", preprocess_text_udf("Complete_Content"))
df = df.withColumn("stemmed_text", preprocess_text_udf_stemmed("Complete_Content"))


In [0]:
# create a new column "Month" by splitting the "Review Date" column on dash and accessing the second element
df = df.withColumn("Month", split(col("Review Date"), "-")[1])

In [0]:
# Applying Lemmatization and Stemming to genre column
df = df.withColumn("genre_lemmatized", preprocess_text_udf("Movie Genre"))
df = df.withColumn("genre_stemmed", preprocess_text_udf_stemmed("Movie Genre"))

### Feature Engineering

Count vectorizer was used for engineer features of the text into a Bag of Words. The Bag of Words was used to train and evaluate the machine learning models.

#### Generating vector of reviews

In [0]:
# Using Count Vectorizer for feature engineering on the lemmatized review text
cv = CountVectorizer(inputCol="lemmatized_text", outputCol="lemmatized_text_vector")
cv_model = cv.fit(df)
count_vectorized_df = cv_model.transform(df)

In [0]:
# Using Count Vectorizer for feature engineering on the stemmed review text
from pyspark.ml.feature import IDF
cv = CountVectorizer(inputCol="stemmed_text", outputCol="stemmed_text_vector")
cv_model = cv.fit(count_vectorized_df)
count_vectorized_df = cv_model.transform(count_vectorized_df)

####Generating Vectors of Genres

In [0]:
# Using Count Vectorizer for feature engineering on the lemmatized genre
cv = CountVectorizer(inputCol="genre_lemmatized", outputCol="genre_lemmatized_vector")
cv_model = cv.fit(count_vectorized_df)
count_vectorized_df = cv_model.transform(count_vectorized_df)

In [0]:
# Using Count Vectorizer for feature engineering on the stemmed genre
cv = CountVectorizer(inputCol="genre_stemmed", outputCol="genre_stemmed_vector")
cv_model = cv.fit(count_vectorized_df)
count_vectorized_df = cv_model.transform(count_vectorized_df)

####Generating vectors for month

In [0]:
count_vectorized_df = count_vectorized_df.withColumn("Month_Vector", split(count_vectorized_df["Month"], ","))

In [0]:
# Using Count Vectorizer for feature engineering on the month text
cv = CountVectorizer(inputCol="Month_Vector", outputCol="Month_vectorized")
cv_model = cv.fit(count_vectorized_df)
count_vectorized_df = cv_model.transform(count_vectorized_df)

#### Combining vectors of review, month and genre

The generated vectors for the review, month and genre are combined into one vector using the VectorAssembler

In [0]:

#from pyspark.ml.feature import VectorAssembler

# Create  for 2 VectorAssembler objects each for lemmaztized or stemmed texts in combination with month and genre.
assembler = VectorAssembler(inputCols=["lemmatized_text_vector", "Month_vectorized", "genre_lemmatized_vector"], outputCol="combined_vectors")
assembler1 = VectorAssembler(inputCols=["stemmed_text_vector", "Month_vectorized", "genre_lemmatized_vector"], outputCol="combined_vectors_stem")

# Apply the created VectorAssembler objects to the DataFrame
assembled_df = assembler.transform(count_vectorized_df)
assembled_df = assembler1.transform(assembled_df)


In [0]:
#assembled_df.display()

_c0,Unnamed: 0,Movie Title,Movie Rating,Movie Genre,Review Date,Review Title,Review Content,ReviewerRating,Manual_Combined,Rating_scaled,Complete_Content,lemmatized_text,stemmed_text,Month,genre_lemmatized,genre_stemmed,lemmatized_text_vector,stemmed_text_vector,genre_lemmatized_vector,genre_stemmed_vector,Month_Vector,Month_vectorized,combined_vectors,combined_vectors_stem
0,0,Lightyear,6.1,Animation Action Adventure,17-Jul-22,Really don't get the hate,It wasn't the best Pixar film but it definitely wasn't the worst. Why it gets such a low score on here I'll never know. I'll admit the trailer did make it look better than it actually was but it was still decent and funny.,7,0,1,Really don't get the hate It wasn't the best Pixar film but it definitely wasn't the worst. Why it gets such a low score on here I'll never know. I'll admit the trailer did make it look better than it actually was but it was still decent and funny.,"List(really, dont, get, hate, wasnt, best, pixar, film, definitely, wasnt, bad, get, low, score, ill, never, know, ill, admit, trailer, make, look, good, actually, still, decent, funny)","List(realli, dont, get, hate, wasnt, best, pixar, film, definit, wasnt, worst, get, low, score, ill, never, know, ill, admit, trailer, make, look, better, actual, still, decent, funni)",Jul,"List(animation, action, adventure)","List(anim, action, adventur)","Map(vectorType -> sparse, length -> 11604, indices -> List(1, 4, 6, 11, 14, 21, 27, 33, 40, 42, 59, 75, 106, 117, 140, 151, 154, 203, 236, 360, 396, 518, 693, 1054), values -> List(1.0, 1.0, 1.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0, 1.0))","Map(vectorType -> sparse, length -> 9578, indices -> List(1, 9, 10, 11, 25, 38, 39, 43, 45, 63, 81, 83, 137, 139, 161, 175, 233, 273, 335, 397, 403, 523, 552, 884), values -> List(1.0, 1.0, 1.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0, 1.0, 1.0))","Map(vectorType -> sparse, length -> 17, indices -> List(1, 4, 7), values -> List(1.0, 1.0, 1.0))","Map(vectorType -> sparse, length -> 17, indices -> List(1, 4, 7), values -> List(1.0, 1.0, 1.0))",List(Jul),"Map(vectorType -> sparse, length -> 12, indices -> List(4), values -> List(1.0))","Map(vectorType -> sparse, length -> 11633, indices -> List(1, 4, 6, 11, 14, 21, 27, 33, 40, 42, 59, 75, 106, 117, 140, 151, 154, 203, 236, 360, 396, 518, 693, 1054, 11608, 11617, 11620, 11623), values -> List(1.0, 1.0, 1.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0))","Map(vectorType -> sparse, length -> 9607, indices -> List(1, 9, 10, 11, 25, 38, 39, 43, 45, 63, 81, 83, 137, 139, 161, 175, 233, 273, 335, 397, 403, 523, 552, 884, 9582, 9591, 9594, 9597), values -> List(1.0, 1.0, 1.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0))"
1,1,Lightyear,6.1,Animation Action Adventure,19-Nov-22,"It's an okay product, but wrong I.P.","""In 1995, Andy got a toy. That toy was based on a movie. This is that movie. "" I think I would've gotten more out of this if they'd left out that opening text. ""Lightyear"" is a decent space adventure movie, the visuals are pretty great and Chris Evans gives a strong performance. But the idea that the Tim Allen Buzz we all know and love came from this movie . . . Eh, I just couldn't get there. There's a bigger ""Top Gun"" vibe going on here than anything ""Toy Story"" related and ""Maverick"" did a much better job of living with failure. I admit I walked into this with low expectations (really, was anyone asking for this movie?) but the Toy Story association is totally forced.",6,0,0,"It's an okay product, but wrong I.P. ""In 1995, Andy got a toy. That toy was based on a movie. This is that movie. "" I think I would've gotten more out of this if they'd left out that opening text. ""Lightyear"" is a decent spa

In [0]:
# Create a CountVectorizer object for the month
assembled_df = assembled_df.withColumn("Manual_Combined_array", split(assembled_df["Manual_Combined"], ","))
cv = CountVectorizer(inputCol="Manual_Combined_array", outputCol="Manual_Combined_vector")

# Fit the CountVectorizer model on the dataframe
cv_model = cv.fit(assembled_df)

# Transform the dataframe to add the count vectorized features column
count_vectorized_df = cv_model.transform(assembled_df)

In [0]:
# Creating the test dataframe which contains only relevant columsn for the ML model
df_test = count_vectorized_df.select(count_vectorized_df["combined_vectors"],count_vectorized_df["combined_vectors_stem"],count_vectorized_df["Manual_Combined"])


In [0]:
#from pyspark.sql.functions import col
# Function to increase the values of a column by 1. 
def increment_column(df, column_name):
    incremented_col = col(column_name) + 1
    new_df = df.withColumn(column_name, incremented_col) 
    return new_df

In [0]:
# Applying the column incrementer to increase the values of ratings by 1. 
df_test2 = increment_column(df_test, "Manual_Combined")

In [0]:
# This is used to create a sample dataset containing the top 10 rows.
# This sample dataset allowed for testing of model within a short time to be sure there are not errors
df_random = df_test2.limit(10)
(training1, testing1) = df_random.randomSplit([0.7, 0.3])
df_random.show()

+--------------------+---------------------+---------------+
|    combined_vectors|combined_vectors_stem|Manual_Combined|
+--------------------+---------------------+---------------+
|(11633,[1,4,6,11,...| (9607,[1,9,10,11,...|              1|
|(11633,[0,4,5,11,...| (9607,[0,4,10,11,...|              1|
|(11633,[1,2,4,5,6...| (9607,[1,2,4,5,7,...|              0|
|(11633,[0,1,2,4,5...| (9607,[0,1,2,4,5,...|              1|
|(11633,[0,1,3,4,5...| (9607,[0,1,3,4,5,...|              1|
|(11633,[0,2,3,4,6...| (9607,[0,2,3,5,6,...|              0|
|(11633,[0,4,7,9,1...| (9607,[0,5,6,7,19...|              2|
|(11633,[5,28,44,5...| (9607,[4,23,35,48...|              1|
|(11633,[1,4,13,14...| (9607,[1,5,10,15,...|              2|
|(11633,[0,1,5,6,7...| (9607,[0,1,4,6,7,...|              1|
+--------------------+---------------------+---------------+



In [0]:
# Splitting the data into train and test datasets.
(training, testing) = df_test2.randomSplit([0.7, 0.3])

## Models

Three machine learning classification models were used:
- Naive Bayes
- Logistic Regression
- Random Forest Classifier

All models are evaluated using F1-score, Recall and Precision. F1-score is the primary metric.

#### Naive Bayes
Naives Bayes model was investigated for lemmatized and stemmed vectors.
The smoothing hyperparameter was tuned as follows:

smoothing = [0.00, 0.10, 0.15, 0.20, 0.50, 1.00, 10.00, 20.00]

Because we have 3 classes of labels (positive, neutral and negative), we use the MulticlassClassificationEvaluator.

##### Naive Bayes with Lemmatized Text

In [0]:
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


# Create Naive Bayes object
nb = NaiveBayes(labelCol="Manual_Combined", featuresCol="combined_vectors", modelType="multinomial")

# Create MulticlassClassificationEvaluator
nbevaluator = MulticlassClassificationEvaluator(labelCol="Manual_Combined", metricName="weightedFMeasure")

# set hyperparameters
params_nb = [0.00, 0.10, 0.15, 0.20, 0.50, 1.00, 10.00, 20.00]

# create list variables to store evaluation metrics
f1_scores_nb =[]
recall_scores_nb = []
precision_scores_nb = []


for i in params_nb:
    # Creating the  ParamGrid for Cross Validation
    nbparamGrid = (ParamGridBuilder()
                   .addGrid(nb.smoothing, [i])
                   .build())


    # Creating and applying 5 fold cross-validation
    nbcv = CrossValidator(estimator = nb,
                        estimatorParamMaps = nbparamGrid,
                        evaluator = nbevaluator,
                        numFolds = 5)
    nbcvModel = nbcv.fit(training)
    prediction = nbcvModel.transform(testing).cache()


    # Evaluating the models using defined metrices (F1 score, precision and recall)
    evaluator_f1_nb = MulticlassClassificationEvaluator(labelCol="Manual_Combined", predictionCol="prediction", metricName="weightedFMeasure")
    f1_score = evaluator_f1_nb.evaluate(prediction)
    f1_scores_nb.append(f1_score)

    evaluator_recall_nb = MulticlassClassificationEvaluator(labelCol="Manual_Combined", predictionCol="prediction", metricName="weightedRecall")
    recall_score = evaluator_recall_nb.evaluate(prediction, {evaluator_recall_nb.metricName: "weightedRecall"})
    recall_scores_nb.append(recall_score)

    evaluator_precision_nb = MulticlassClassificationEvaluator(labelCol="Manual_Combined", predictionCol="prediction", metricName="weightedPrecision")
    precision_score = evaluator_precision_nb.evaluate(prediction, {evaluator_precision_nb.metricName: "weightedPrecision"})
    precision_scores_nb.append(precision_score)



In [0]:
#Storing the metrices into a dataframe and displaying the dataframe
nb_total_result = []
for i in range(len(params_nb)):
    templist = []
    templist.append(params_nb[i])
    templist.append(f1_scores_nb[i])
    templist.append(recall_scores_nb[i])
    templist.append(precision_scores_nb[i])
    nb_total_result.append(templist)

schema = StructType([
    StructField("Smoothing", StringType(), True),
    StructField("f1_score", StringType(), True),
    StructField("recall_score", StringType(), True),
    StructField("precision_score", StringType(), True)
])

df_nb_results = spark.createDataFrame(data=nb_total_result, schema=schema)
df_nb_results.display()

Smoothing,f1_score,recall_score,precision_score
0.0,0.2091387177594074,0.3385579937304075,0.44153747794854875
0.1,0.6348950270573637,0.6394984326018809,0.6340436589313605
0.15,0.6412603501037722,0.6489028213166144,0.6415334926306713
0.2,0.6489959761427255,0.658307210031348,0.6478869572854639
0.5,0.66389340512258,0.6927899686520376,0.6520336443600991
1.0,0.6373555450634558,0.6959247648902821,0.653460382218013
10.0,0.3314252569950992,0.4921630094043887,0.5604659671884735
20.0,0.317778395037234,0.48589341692789967,0.23609241261386973


##### Naive Bayes with Stemmed Text

In [0]:
# Create Naive Bayes object
nb = NaiveBayes(labelCol="Manual_Combined", featuresCol="combined_vectors_stem", modelType="multinomial")

# Create MulticlassClassificationEvaluator
nbevaluator = MulticlassClassificationEvaluator(labelCol="Manual_Combined", metricName="weightedFMeasure")
# set hyperparameters
params_nb = [0.00, 0.10, 0.15, 0.20, 0.50, 1.00, 10.00, 20.00]
# create list variables to store evaluation metrics
f1_scores_nb =[]
recall_scores_nb = []
precision_scores_nb = []


for i in params_nb:
    # Creating the  ParamGrid for Cross Validation
    nbparamGrid = (ParamGridBuilder()
                   .addGrid(nb.smoothing, [i])
                   .build())


    # Create and apply 5 fold cross-validation
    nbcv = CrossValidator(estimator = nb,
                        estimatorParamMaps = nbparamGrid,
                        evaluator = nbevaluator,
                        numFolds = 5)
    nbcvModel = nbcv.fit(training)
    prediction = nbcvModel.transform(testing).cache()

    # Evaluating the models using defined metrices (F1 score, precision and recall)
    evaluator_f1_nb = MulticlassClassificationEvaluator(labelCol="Manual_Combined", predictionCol="prediction", metricName="weightedFMeasure")
    f1_score = evaluator_f1_nb.evaluate(prediction)
    f1_scores_nb.append(f1_score)

    evaluator_recall_nb = MulticlassClassificationEvaluator(labelCol="Manual_Combined", predictionCol="prediction", metricName="weightedRecall")
    recall_score = evaluator_recall_nb.evaluate(prediction, {evaluator_recall_nb.metricName: "weightedRecall"})
    recall_scores_nb.append(recall_score)

    evaluator_precision_nb = MulticlassClassificationEvaluator(labelCol="Manual_Combined", predictionCol="prediction", metricName="weightedPrecision")
    precision_score = evaluator_precision_nb.evaluate(prediction, {evaluator_precision_nb.metricName: "weightedPrecision"})
    precision_scores_nb.append(precision_score)

In [0]:
#Storing the metrices into a dataframe and displaying the dataframe
nb_total_result = []
for i in range(len(params_nb)):
    templist = []
    templist.append(params_nb[i])
    templist.append(f1_scores_nb[i])
    templist.append(recall_scores_nb[i])
    templist.append(precision_scores_nb[i])
    nb_total_result.append(templist)

from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType

schema = StructType([
    StructField("Smoothing", StringType(), True),
    StructField("f1_score", StringType(), True),
    StructField("recall_score", StringType(), True),
    StructField("precision_score", StringType(), True)
])

df_nb_results = spark.createDataFrame(data=nb_total_result, schema=schema)
df_nb_results.display()

Smoothing,f1_score,recall_score,precision_score
0.0,0.2393391330711868,0.35736677115987464,0.5077755240027045
0.1,0.6359807569530576,0.6426332288401254,0.6328574062853981
0.15,0.6400759903354278,0.6489028213166144,0.6356871292002383
0.2,0.6524918405442343,0.664576802507837,0.645924394784469
0.5,0.649781388962519,0.6802507836990596,0.6362276530176258
1.0,0.6284662070922841,0.6833855799373041,0.6177341006771211
10.0,0.35776537908333605,0.5047021943573667,0.5181470828022552
20.0,0.3246595389009911,0.4890282131661442,0.5597188541235386


%md
The best F1-score was 0.0.6639 for regParam = 0.5 using lemmatized text. The corresponding recall and precision were 0.6928 and 0.65203 respectively. Using stemmed text, best F1-score was 0.6524 for regParam = 0.2. The corresponding recall and precision were 0.6646 and 0.6459 respectively.

Overall, the model performed better with review data featurizated using lemmatization than snowball stemming.

### Logistic Regression
Logistic Regression model was investigated for lemmatized and stemmed vectors. The regParam hyperparameter was tuned while elasticNetParam was kept at the default value of zero.

regParam = [0.0001, 0.001, 0.005, 0.01, 0.1, 1.0, 10.0]

Because we have 3 classes of labels (positive, neutral and negative), we use the MulticlassClassificationEvaluator.

##### Logistic Regression with Lemmatized Text

In [0]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Create LogisticRegression object
lr = LogisticRegression(labelCol="Manual_Combined", featuresCol="combined_vectors")

# Create MulticlassClassificationEvaluator
lrevaluator = MulticlassClassificationEvaluator(labelCol="Manual_Combined", metricName="weightedFMeasure")

# set hyperparameters
params_lr = [0.0001, 0.001, 0.005, 0.01, 0.1, 1.0, 10.0]
# create list variables to store evaluation metrics
f1_scores_lr =[]
recall_scores_lr = []
precision_scores_lr = []


for i in params_lr:
     # Creating the  ParamGrid for Cross Validation
    lrparamGrid = (ParamGridBuilder()
                   .addGrid(lr.regParam, [i])
                   .build())


    # Create and apply 5 fold cross-validation
    lrcv = CrossValidator(estimator = lr,
                        estimatorParamMaps = lrparamGrid,
                        evaluator = lrevaluator,
                        numFolds = 5)
    lrcvModel = lrcv.fit(training)

    prediction = lrcvModel.transform(testing).cache()

     # Evaluating the models using defined metrices (F1 score, precision and recall)
    evaluator_f1_lr = MulticlassClassificationEvaluator(labelCol="Manual_Combined", predictionCol="prediction", metricName="weightedFMeasure")
    f1_score = evaluator_f1_lr.evaluate(prediction)
    f1_scores_lr.append(f1_score)

    evaluator_recall_lr = MulticlassClassificationEvaluator(labelCol="Manual_Combined", predictionCol="prediction", metricName="weightedRecall")
    recall_score = evaluator_recall_lr.evaluate(prediction, {evaluator_recall_lr.metricName: "weightedRecall"})
    recall_scores_lr.append(recall_score)

    evaluator_precision_lr = MulticlassClassificationEvaluator(labelCol="Manual_Combined", predictionCol="prediction", metricName="weightedPrecision")
    precision_score = evaluator_precision_lr.evaluate(prediction, {evaluator_precision_lr.metricName: "weightedPrecision"})
    precision_scores_lr.append(precision_score)


In [0]:
#Storing the metrices into a dataframe and displaying the dataframe
lr_total_result = []
for i in range(len(params_lr)):
    templist = []
    templist.append(params_lr[i])
    templist.append(f1_scores_lr[i])
    templist.append(recall_scores_lr[i])
    templist.append(precision_scores_lr[i])
    lr_total_result.append(templist)

from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType

schema = StructType([
    StructField("Alpha", StringType(), True),
    StructField("f1_score", StringType(), True),
    StructField("recall_score", StringType(), True),
    StructField("precision_score", StringType(), True)
])

df_lr_results = spark.createDataFrame(data=lr_total_result, schema=schema)
df_lr_results.display()

Alpha,f1_score,recall_score,precision_score
1.0E-4,0.6675331242182356,0.7021943573667712,0.6743944059473297
0.001,0.6587970466864819,0.6990595611285266,0.6639461256712749
0.005,0.6606767404234214,0.7021943573667712,0.6688037049947837
0.01,0.6606767404234214,0.7021943573667712,0.6688037049947837
0.1,0.6698721556438328,0.7178683385579937,0.682706220869837
1.0,0.6597611737711275,0.7147335423197492,0.7079347473598623
10.0,0.4541024178911041,0.561128526645768,0.575231124807396


##### Logistic Regression with Stemmed Text

In [0]:
# Create LogisticRegression object
lr = LogisticRegression(labelCol="Manual_Combined", featuresCol="combined_vectors_stem")

# Create MulticlassClassificationEvaluator
lrevaluator = MulticlassClassificationEvaluator(labelCol="Manual_Combined", metricName="weightedFMeasure")

# set hyperparameters
params_lr = [0.0001, 0.001, 0.005, 0.01, 0.1, 1.0, 10.0]


# create list variables to store evaluation metrics
f1_scores_lr =[]
recall_scores_lr = []
precision_scores_lr = []


for i in params_lr:
    # Creating the  ParamGrid for Cross Validation
    lrparamGrid = (ParamGridBuilder()
                   .addGrid(lr.regParam, [i])
                   .build())


    # Create and apply 5 fold cross-validation
    lrcv = CrossValidator(estimator = lr,
                        estimatorParamMaps = lrparamGrid,
                        evaluator = lrevaluator,
                        numFolds = 5)
    lrcvModel = lrcv.fit(training)

    prediction = lrcvModel.transform(testing).cache()

     # Evaluating the models using defined metrices (F1 score, precision and recall)
    evaluator_f1_lr = MulticlassClassificationEvaluator(labelCol="Manual_Combined", predictionCol="prediction", metricName="weightedFMeasure")
    f1_score = evaluator_f1_lr.evaluate(prediction)
    f1_scores_lr.append(f1_score)

    evaluator_recall_lr = MulticlassClassificationEvaluator(labelCol="Manual_Combined", predictionCol="prediction", metricName="weightedRecall")
    recall_score = evaluator_recall_lr.evaluate(prediction, {evaluator_recall_lr.metricName: "weightedRecall"})
    recall_scores_lr.append(recall_score)

    evaluator_precision_lr = MulticlassClassificationEvaluator(labelCol="Manual_Combined", predictionCol="prediction", metricName="weightedPrecision")
    precision_score = evaluator_precision_lr.evaluate(prediction, {evaluator_precision_lr.metricName: "weightedPrecision"})
    precision_scores_lr.append(precision_score)

In [0]:
#Storing the metrices into a dataframe and displaying the dataframe
lr_total_result = []
for i in range(len(params_lr)):
    templist = []
    templist.append(params_lr[i])
    templist.append(f1_scores_lr[i])
    templist.append(recall_scores_lr[i])
    templist.append(precision_scores_lr[i])
    lr_total_result.append(templist)

from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType

schema = StructType([
    StructField("Alpha", StringType(), True),
    StructField("f1_score", StringType(), True),
    StructField("recall_score", StringType(), True),
    StructField("precision_score", StringType(), True)
])

df_lr_results = spark.createDataFrame(data=lr_total_result, schema=schema)
df_lr_results.display()

Alpha,f1_score,recall_score,precision_score
1.0E-4,0.625734683329322,0.658307210031348,0.6312730582111626
0.001,0.6247058907157429,0.6614420062695925,0.6264637919017191
0.005,0.6346392681187767,0.677115987460815,0.6410321441556098
0.01,0.640840291625489,0.6833855799373041,0.6475013228557814
0.1,0.6581156254694192,0.7021943573667712,0.6633075437934373
1.0,0.6479756218056709,0.6990595611285266,0.6606867113196228
10.0,0.46426577815519726,0.567398119122257,0.5804479738221059


The best F1-score was 0.6698 for smoothing = 0.1 using lemmatized text. The corresponding recall and precision were 0.7178 and 0.6817 respectively. Using stemmed text, best F1-score was 0.6581 for smoothing = 0.1. The corresponding recall and precision were 0.7021 and 0.6633 respectively.

Overall, the model performed better with review data featurizated using lemmatization than snowball stemming.

### RandomForestClassifier
Random Forest Classifier model was investigated for lemmatized and stemmed vectors. Two hyperparameters were investigated as follows:  The regParam hyperparameter was tuned while elasticNetParam was kept at the default value of zero.

- numTrees = [5, 10, 30, 50]
- maxDepth = [5, 10, 15, 20, 30]

Because we have 3 classes of labels (positive, neutral and negative), we use the MulticlassClassificationEvaluator.

##### RandomForestClassifier with Lemmatized Text

In [0]:
from pyspark.sql.types import IntegerType
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Create RandomForestClassifier object
rf = RandomForestClassifier(labelCol="Manual_Combined", featuresCol="combined_vectors")

# Create MulticlassClassificationEvaluator
rf_evaluator = MulticlassClassificationEvaluator(labelCol="Manual_Combined", metricName="weightedFMeasure")

# set hyperparameters
params_rf_trees = [5, 10, 30, 50]
params_rf_maxdepth = [5, 10, 15, 20, 30]

# create list variables to store evaluation metrics
i=0
f1_scores_rf =[]
recall_scores_rf = []
precision_scores_rf = []

#iterating through max depth
for depth in params_rf_maxdepth:
    j=0
    #iterating through number of trees
    for trees in params_rf_trees:
        # Creating the  ParamGrid for Cross Validation
        rfparamGrid = (ParamGridBuilder()
                       .addGrid(rf.numTrees, [trees]) \
        .addGrid(rf.maxDepth, [depth]) \
                       .build())


        # Create and apply 5 fold cross-validation
        rfcv = CrossValidator(estimator = rf,
                            estimatorParamMaps = rfparamGrid,
                            evaluator = rf_evaluator,
                            numFolds = 5)
        rfcvModel = rfcv.fit(training)

        prediction = rfcvModel.transform(testing).cache()

        #Evaluating F1 score
        evaluator_f1_rf = MulticlassClassificationEvaluator(labelCol="Manual_Combined", predictionCol="prediction", metricName="weightedFMeasure")
        f1_score = evaluator_f1_rf.evaluate(prediction)
        f1_scores_rf.append(f1_score)
        
        # Evaluating the models using defined metrices (F1 score, precision and recall)
        #evaluating recall score
        evaluator_recall_rf = MulticlassClassificationEvaluator(labelCol="Manual_Combined", predictionCol="prediction", metricName="weightedRecall")
        recall_score = evaluator_recall_rf.evaluate(prediction, {evaluator_recall_rf.metricName: "weightedRecall"})
        recall_scores_rf.append(recall_score)

        #evaluating precision score
        evaluator_precision_rf = MulticlassClassificationEvaluator(labelCol="Manual_Combined", predictionCol="prediction", metricName="weightedPrecision")
        precision_score = evaluator_precision_rf.evaluate(prediction, {evaluator_precision_rf.metricName: "weightedPrecision"})
        precision_scores_rf.append(precision_score)



In [0]:
#Storing the metrices into a dataframe and displaying the dataframe
x=0
for i in range(5):
    for j in range(4):
        print("\n\nMax depth is : " + str(params_rf_maxdepth[i]))
        print("Number of trees is: " + str(params_rf_trees[j]))
        print("f1 is: " + str(f1_scores_rf[x]))
        print("recall is: " + str(recall_scores_rf[x]))
        print("precision is: " + str(precision_scores_rf[x]))
        x+=1



Max depth is : 5
Number of trees is: 5
f1 is: 0.4364316358451198
recall is: 0.5397923875432525
precision is: 0.4907203523120478


Max depth is : 5
Number of trees is: 10
f1 is: 0.41653858427414653
recall is: 0.5432525951557093
precision is: 0.5466877287999599


Max depth is : 5
Number of trees is: 30
f1 is: 0.36269080687303384
recall is: 0.5190311418685121
precision is: 0.48415483017559136


Max depth is : 5
Number of trees is: 50
f1 is: 0.3620261998277072
recall is: 0.5190311418685121
precision is: 0.5928046972016927


Max depth is : 10
Number of trees is: 5
f1 is: 0.4746413685076859
recall is: 0.5501730103806228
precision is: 0.4806128696939924


Max depth is : 10
Number of trees is: 10
f1 is: 0.5099986683663995
recall is: 0.5916955017301038
precision is: 0.5925236447520185


Max depth is : 10
Number of trees is: 30
f1 is: 0.45034320171058295
recall is: 0.560553633217993
precision is: 0.5097579212059032


Max depth is : 10
Number of trees is: 50
f1 is: 0.4473206770784625
recall is:

##### RandomForestClassifier with Stemmed Text

In [0]:
from pyspark.sql.types import IntegerType
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


# Create RandomForestClassifier object
rf = RandomForestClassifier(labelCol="Manual_Combined", featuresCol="combined_vectors_stem")

# Create MulticlassClassificationEvaluator
rf_evaluator = MulticlassClassificationEvaluator(labelCol="Manual_Combined", metricName="weightedFMeasure")

# set hyperparameters
params_rf_trees1 = [5, 10, 30, 50]
params_rf_maxdepth1 = [5, 10, 15, 20, 30]

# create list variables to store evaluation metrics
i=0
f1_scores_rf1 =[]
recall_scores_rf1 = []
precision_scores_rf1 = []

#iterating through max depth
for depth in params_rf_maxdepth1:
    j=0
    #iterating through number of trees
    for trees in params_rf_trees1:
        # Creating the  ParamGrid for Cross Validation
        rfparamGrid = (ParamGridBuilder()
                       .addGrid(rf.numTrees, [trees]) \
        .addGrid(rf.maxDepth, [depth]) \
                       .build())


        # Create and apply 5 fold cross-validation
        rfcv = CrossValidator(estimator = rf,
                            estimatorParamMaps = rfparamGrid,
                            evaluator = rf_evaluator,
                            numFolds = 5)
        rfcvModel = rfcv.fit(training)

        prediction = rfcvModel.transform(testing).cache()
        
        # Evaluating the models using defined metrices (F1 score, precision and recall)
        #Evaluating F1 score
        evaluator_f1_rf = MulticlassClassificationEvaluator(labelCol="Manual_Combined", predictionCol="prediction", metricName="weightedFMeasure")
        f1_score = evaluator_f1_rf.evaluate(prediction)
        f1_scores_rf1.append(f1_score)
        
        #evaluating recall score
        evaluator_recall_rf = MulticlassClassificationEvaluator(labelCol="Manual_Combined", predictionCol="prediction", metricName="weightedRecall")
        recall_score = evaluator_recall_rf.evaluate(prediction, {evaluator_recall_rf.metricName: "weightedRecall"})
        recall_scores_rf1.append(recall_score)

        #evaluating precision score
        evaluator_precision_rf = MulticlassClassificationEvaluator(labelCol="Manual_Combined", predictionCol="prediction", metricName="weightedPrecision")
        precision_score = evaluator_precision_rf.evaluate(prediction, {evaluator_precision_rf.metricName: "weightedPrecision"})
        precision_scores_rf1.append(precision_score)



In [0]:
#Storing the metrices into a dataframe and displaying the dataframe
x=0
for i in range(5):
    for j in range(4):
        print("\n\nMax depth is : " + str(params_rf_maxdepth1[i]))
        print("Number of trees is: " + str(params_rf_trees1[j]))
        print("f1 is: " + str(f1_scores_rf1[x]))
        print("recall is: " + str(recall_scores_rf1[x]))
        print("precision is: " + str(precision_scores_rf1[x]))
        x+=1



Max depth is : 5
Number of trees is: 5
f1 is: 0.45850004749518114
recall is: 0.552901023890785
precision is: 0.4590273604481081


Max depth is : 5
Number of trees is: 10
f1 is: 0.4536799824607166
recall is: 0.5631399317406144
precision is: 0.6514716809604637


Max depth is : 5
Number of trees is: 30
f1 is: 0.46156528340828673
recall is: 0.5733788395904437
precision is: 0.5702539214231471


Max depth is : 5
Number of trees is: 50
f1 is: 0.45232717660443006
recall is: 0.5699658703071673
precision is: 0.5867004293735549


Max depth is : 10
Number of trees is: 5
f1 is: 0.5149920711858278
recall is: 0.590443686006826
precision is: 0.4880174075447132


Max depth is : 10
Number of trees is: 10
f1 is: 0.5454026447879204
recall is: 0.6177474402730376
precision is: 0.693188282138794


Max depth is : 10
Number of trees is: 30
f1 is: 0.5555769122456392
recall is: 0.6382252559726962
precision is: 0.5820253040698609


Max depth is : 10
Number of trees is: 50
f1 is: 0.5486077252288856
recall is: 0.

Below is the best performing hyperparameter for lemmatized text:
- Hyperparameters
  - Max depth is : 30
  - Number of trees is: 50
- Metric
  - f1 is: 0.6055
  - recall is: 0.6712
  - precision is: 0.5974

Below is the best performing hyperparameter for lemmatized text:
- Hyperparameters
  - Max depth is : 30
  - Number of trees is: 50
- Metric
  - f1 is: 0.6315
  - recall is: 0.7031
  - precision is: 0.5822

Overall, the model performed better with review data featurizated using snowball stemming than lemmatization.

### Conclusion

Across the entire models, the Naive Bayes Model ourperformed all other models with an  F1-score was 0.0.6639 for regParam = 0.5 using lemmatized text. The corresponding recall and precision were 0.6928 and 0.65203 respectively.

For both Naive Bayes and LogisticRegression, using review data featurized using lemmatization outperformed models investigated using lemmatization.

It is worth noting that random seed were not used in running the models. As such, subsequent runs will yield slightly different values